<br />
<div align="center">
  <a href="https://deepwok.github.io/">
    <img src="../imgs/deepwok.png" alt="Logo" width="160" height="160">
  </a>

  <h1 align="center">Lab 4 for Advanced Deep Learning Systems (ADLS) - Hardware Stream</h1>

  <p align="center">
    ELEC70109/EE9-AML3-10/EE9-AO25
    <br />
		Written by
    <a href="https://aaron-zhao123.github.io/">Aaron Zhao, Pedro Gimenes </a>
  </p>
</div>

# General introduction

In this lab, you will learn how to emit SystemVerilog code for a neural network that's been transformed and optimized by MASE. Then, you'll design some hardware for a new Pytorch layer, and simulate the hardware using your new module.

# The Hardware Emit pass

The `emit_verilog` transform pass generates a top-level RTL file and testbench file according to the `MaseGraph`, which includes a hardware implementation of each layer in the network. This top-level file instantiates modules from the `components` library in MASE and/or modules generated using [HLS](https://en.wikipedia.org/wiki/High-level_synthesis), when internal components are not available. The hardware can then be simulated using [Verilator](https://www.veripool.org/verilator/), or deployed on an FPGA.

First, add Machop to your system PATH (if you haven't already done so) and import the required libraries.

In [1]:
import os, sys
import torch
torch.manual_seed(0)

from chop.ir.graph.mase_graph import MaseGraph

from chop.passes.graph.analysis import (
    init_metadata_analysis_pass,
    add_common_metadata_analysis_pass,
    add_hardware_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    report_node_type_analysis_pass,
)

from chop.passes.graph.transforms import (
    emit_verilog_top_transform_pass,
    emit_internal_rtl_transform_pass,
    emit_bram_transform_pass,
    emit_cocotb_transform_pass,
    quantize_transform_pass,
)

from chop.tools.logger import set_logging_verbosity

set_logging_verbosity("debug")

import toml
import torch
import torch.nn as nn

# TO DO: remove
import os
os.environ["PATH"] = "/opt/homebrew/bin:" + os.environ["PATH"]
!verilator

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Set logging level to debug


Usage:
        verilator --help
        verilator --version
        verilator --binary -j 0 [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --cc [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --sc [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --lint-only -Wall [source_files.v]...



Now, define the neural network. We're using a model which can be used to perform digit classification on the MNIST dataset.

In [2]:
class MLP(torch.nn.Module):
    """
    Toy FC model for digit recognition on MNIST
    """

    def __init__(self) -> None:
        super().__init__()

        self.fc1 = nn.Linear(4, 4)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1, end_dim=-1)
        x=self.fc1(x)
        # x = torch.nn.functional.relu(self.fc1(x),inplace=False)
        # x = torch.nn.functional.leaky_relu(self.fc1(x),negative_slope=0.5,inplace=False)
        return x

In [3]:
import numpy as np
def float_to_fixed(x, scaling_factor, total_bits=8):
    """
    Convert a floating-point numpy array to fixed-point representation.

    Parameters:
      x            : numpy array of floats.
      scaling_factor: Factor to scale the float values (2^frac_bits).
      total_bits   : Total number of bits (default is 8).

    Returns:
      A numpy array of type np.int8 representing the fixed-point values.
    """
    # Scale the floating-point values.
    x_scaled = x * scaling_factor

    # Round to the nearest integer.
    x_fixed = np.trunc(x_scaled)

    # Define the representable range for signed 8-bit integers.
    min_val = -2**(total_bits - 1)       # -128 for 8 bits
    max_val = 2**(total_bits - 1) - 1      #  127 for 8 bits

    # Clip values that exceed the representable range.
    x_fixed = np.clip(x_fixed, min_val, max_val)

    # Return as np.int8.
    return x_fixed.astype(np.int8)


In [4]:

frac_bits = 8  # for example
scale = 2 ** frac_bits



In [5]:
# import numpy as np

# weight_np = np.array([
#     [-0.0037,  0.2682, -0.4115, -0.3680],
#     [-0.1926,  0.1341, -0.0099,  0.3964],
#     [-0.0444,  0.1323, -0.1511, -0.0983],
#     [-0.4777, -0.3311, -0.2061,  0.0185],
#     [ 0.1977,  0.3000, -0.3390, -0.2177],
#     [ 0.1816,  0.4152, -0.1029,  0.3742],
#     [-0.0806,  0.0529,  0.4527, -0.4638],
#     [-0.3148, -0.1266, -0.1949,  0.4320]
# ])  # shape (8,4)

# bias_np = np.array([-0.3241, -0.2302, -0.3493, -0.4683,
#                     -0.2919,  0.4298,  0.2231,  0.2423])  # shape (8,)

# # Convert them to PyTorch tensors of the correct shape and dtype
# weight_torch = torch.tensor(weight_np, dtype=torch.float32)
# bias_torch   = torch.tensor(bias_np,   dtype=torch.float32)


# model = MLP()

# # Overwrite the linear layer’s weight/bias data
# # Note: PyTorch layers store weight/bias in .weight and .bias
# with torch.no_grad():
#     model.fc1.weight.copy_(weight_torch)
#     model.fc1.bias.copy_(bias_torch)

# # Input as a NumPy array, shape = (1, 4)
# input_np = np.array([[0.4963, 0.7682, 0.0885, 0.1320]])

# # Convert to PyTorch tensor
# input_torch = torch.tensor(input_np, dtype=torch.float32)

# # Forward pass
# output = model(input_torch)
# print("Output:\n", output)

In [6]:
import torch

print("Supported quantized engines:", torch.backends.quantized.supported_engines)
print("Current quantized engine:   ", torch.backends.quantized.engine)

Supported quantized engines: ['qnnpack', 'none', 'onednn']
Current quantized engine:    onednn


Now, we'll generate a MaseGraph and add metadata. 

In [7]:
mlp = MLP()
mg = MaseGraph(model=mlp)

# Provide a dummy input for the graph so it can use for tracing
batch_size = 1
x = torch.randn((batch_size, 2, 2))
print(x)
dummy_in = {"x": x}

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(
    mg, {"dummy_in": dummy_in, "add_value": False}
)

DEBUG    graph():
    %x : [num_users=1] = placeholder[target=x]
    %flatten : [num_users=1] = call_function[target=torch.flatten](args = (%x,), kwargs = {start_dim: 1, end_dim: -1})
    %fc1 : [num_users=1] = call_module[target=fc1](args = (%flatten,), kwargs = {})
    return fc1


tensor([[[-0.5966,  0.1820],
         [-0.8567,  1.1006]]])
Hellos in add_common_metadata
sigoyi in add_common_metadata
nihhhhhhhhh in graph_iterator_for_mase_ops in add_common_metadata
graph_model:  GraphModule(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
)



def forward(self, x):
    flatten = torch.flatten(x, start_dim = 1, end_dim = -1);  x = None
    fc1 = self.fc1(flatten);  flatten = None
    return fc1
    
# To see more debug info, please use `graph_module.print_readable()`
wocao in add_common_metadata
graph_iterator_for_metadata:  GraphModule(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
)



def forward(self, x):
    flatten = torch.flatten(x, start_dim = 1, end_dim = -1);  x = None
    fc1 = self.fc1(flatten);  flatten = None
    return fc1
    
# To see more debug info, please use `graph_module.print_readable()`
args[i]: tensor([[[-0.5966,  0.1820],
         [-0.8567,  1.1006]]])
get_shape(args[i]) [1, 2, 2]
kwargs:  {'start_dim': 1, 'end_dim': 

Before running `emit_verilog`, we'll quantize the model to fixed precision. Refer back to [lab 3](https://deepwok.github.io/mase/modules/labs_2023/lab3.html) if you've forgotten how this works. Check that the data type for each node is correct after quantization.

In [8]:
config_file = os.path.join(
    os.path.abspath(""),
    "..",
    "..",
    "configs",
    "tests",
    "quantize",
    "fixed.toml",
)
with open(config_file, "r") as f:
    quan_args = toml.load(f)["passes"]["quantize"]
mg, _ = quantize_transform_pass(mg, quan_args)

_ = report_node_type_analysis_pass(mg)

# Update the metadata
for node in mg.fx_graph.nodes:
    for arg, arg_info in node.meta["mase"]["common"]["args"].items():
        if isinstance(arg_info, dict):
            arg_info["type"] = "fixed"
            arg_info["precision"] = [8, 5]
    for result, result_info in node.meta["mase"]["common"]["results"].items():
        if isinstance(result_info, dict):
            result_info["type"] = "fixed"
            result_info["precision"] = [8, 5]

INFO     Inspecting graph [add_common_node_type_analysis_pass]
INFO     
Node name    Fx Node op     Mase type            Mase op      Value type
-----------  -------------  -------------------  -----------  ------------
x            placeholder    placeholder          placeholder  NA
flatten      call_function  implicit_func        flatten      float
fc1          call_module    module_related_func  linear       fixed
output       output         output               output       NA


placeholder
flatten
node_config:  {'name': 'fixed', 'data_in_width': 8, 'data_in_frac_width': 5, 'weight_width': 8, 'weight_frac_width': 5, 'bias_width': 8, 'bias_frac_width': 5, 'data_out_width': 8, 'data_out_frac_width': 5, 'floor': True}
output


At this point, it's important to run the `add_hardware_metadata` analysis pass. This adds all the required metadata which is later used by the `emit_verilog` pass, including:

1. The node's toolchain, which defines whether we use internal Verilog modules from the `components` library or the HLS flow.
2. The Verilog parameters associated with each node.

> **_TASK:_** Read [this page](https://deepwok.github.io/mase/modules/chop/analysis/add_metadata.html#add-hardware-metadata-analysis-pass) for more information on the hardware metadata pass.

In [9]:
mg, _ = add_hardware_metadata_analysis_pass(mg)
for node in mg.nodes:
        mase_op = node.meta["mase"]["common"]["mase_op"]
        print ('mase_op:', mase_op)
        print ("common:",node.meta["mase"]["common"])
        print ("hardware:",node.meta["mase"]["hardware"])

for node in mg.fx_graph.nodes:
        if node.meta["mase"].parameters["hardware"]["is_implicit"]:
            continue
        # Only modules have internal parameters
        if node.meta["mase"].module is None:
            continue
        # print (node.meta["mase"].parameters["hardware"])
        # Only checks the hardware data that contains the key toolchain
        if "INTERNAL" in node.meta["mase"].parameters["hardware"]["toolchain"]:
                for param_name, parameter in node.meta["mase"].module.named_parameters():
                        print ("param_name in CNN.jynb:",param_name)
                        print ("parameter in CNN.jynb:", parameter)

"""
weights and bias in the Conv/linear in Maze
param_data = node.meta["mase"].module.get_parameter(param_name).data
print ("param_data: ", param_data)
"""

mase_op: linear
2222
 I changed max_parallelism add_hardware_metadata_analysis_pass
vp:  {}
arg in add_verilog_param: data_in_0
vp after for:  {'DATA_IN_0_PRECISION_0': 8, 'DATA_IN_0_PRECISION_1': 5, 'DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'DATA_IN_0_PARALLELISM_DIM_0': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_1': 1, 'DATA_IN_0_PARALLELISM_DIM_1': 1}
arg in add_verilog_param: weight
vp after for:  {'DATA_IN_0_PRECISION_0': 8, 'DATA_IN_0_PRECISION_1': 5, 'DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'DATA_IN_0_PARALLELISM_DIM_0': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_1': 1, 'DATA_IN_0_PARALLELISM_DIM_1': 1, 'WEIGHT_PRECISION_0': 8, 'WEIGHT_PRECISION_1': 5, 'WEIGHT_TENSOR_SIZE_DIM_0': 4, 'WEIGHT_PARALLELISM_DIM_0': 2, 'WEIGHT_TENSOR_SIZE_DIM_1': 4, 'WEIGHT_PARALLELISM_DIM_1': 2}
arg in add_verilog_param: bias
vp after for:  {'DATA_IN_0_PRECISION_0': 8, 'DATA_IN_0_PRECISION_1': 5, 'DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'DATA_IN_0_PARALLELISM_DIM_0': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_1': 1, 'DATA_IN_0_PARALLELISM_DIM_1': 1, 'WEIGHT_PR

'\nweights and bias in the Conv/linear in Maze\nparam_data = node.meta["mase"].module.get_parameter(param_name).data\nprint ("param_data: ", param_data)\n'

In [10]:
mg_soft, _ = add_software_metadata_analysis_pass(mg)
print (mg_soft)

Finally, run the emit verilog pass to generate the SystemVerilog files.

In [11]:
mg, _ = emit_verilog_top_transform_pass(mg)
mg, _ = emit_internal_rtl_transform_pass(mg)

INFO     Emitting Verilog...
INFO     Emitting internal components...


Parameter_map in VerilogEmitter: {'fc1_DATA_IN_0_PRECISION_0': 8, 'fc1_DATA_IN_0_PRECISION_1': 5, 'fc1_DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'fc1_DATA_IN_0_PARALLELISM_DIM_0': 2, 'fc1_DATA_IN_0_TENSOR_SIZE_DIM_1': 1, 'fc1_DATA_IN_0_PARALLELISM_DIM_1': 1, 'fc1_WEIGHT_PRECISION_0': 8, 'fc1_WEIGHT_PRECISION_1': 5, 'fc1_WEIGHT_TENSOR_SIZE_DIM_0': 4, 'fc1_WEIGHT_PARALLELISM_DIM_0': 2, 'fc1_WEIGHT_TENSOR_SIZE_DIM_1': 4, 'fc1_WEIGHT_PARALLELISM_DIM_1': 2, 'fc1_BIAS_PRECISION_0': 8, 'fc1_BIAS_PRECISION_1': 5, 'fc1_BIAS_TENSOR_SIZE_DIM_0': 4, 'fc1_BIAS_PARALLELISM_DIM_0': 2, 'fc1_BIAS_TENSOR_SIZE_DIM_1': 1, 'fc1_BIAS_PARALLELISM_DIM_1': 1, 'fc1_DATA_OUT_0_PRECISION_0': 8, 'fc1_DATA_OUT_0_PRECISION_1': 5, 'fc1_DATA_OUT_0_TENSOR_SIZE_DIM_0': 4, 'fc1_DATA_OUT_0_PARALLELISM_DIM_0': 2, 'fc1_DATA_OUT_0_TENSOR_SIZE_DIM_1': 1, 'fc1_DATA_OUT_0_PARALLELISM_DIM_1': 1, 'DATA_IN_0_PRECISION_0': 8, 'DATA_IN_0_PRECISION_1': 5, 'DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'DATA_IN_0_PARALLELISM_DIM_0': 2, 'DATA_IN_0_TENSOR_SI

The generated files should now be found under `top/hardware`. 

> **_TASK:_** Read through `top/hardware/rtl/top.sv` and make sure you understand how our MLP model maps to this hardware design. 

You will notice the following instantiated modules:

* `fixed_linear`: this is found under `components/linear/fixed_linear.sv` and implements each Linear layer in the model.
* `fc<layer number>_weight/bias_source`: these are [BRAM](https://nandland.com/lesson-15-what-is-a-block-ram-bram/) memories which drive the weights and biases into the linear layers for computation.
* `fixed_relu`: found under `components/activations/fixed_relu.sv`, implements the ReLU activation.

As of now, we can't yet run a simulation on the model, as we haven't yet generated the memory components. To do this, run the `emit_bram` transform pass as follows, which will generate the memory initialization files and SystemVerilog modules to drive weights and biases into the linear layers. Finally, the `emit_verilog_tb` transform pass will generate the testbench files.


In [12]:
mg, _ = emit_bram_transform_pass(mg)

INFO     Emitting BRAM...
DEBUG    Emitting DAT file for node: fc1, parameter: weight
DEBUG    ROM module weight successfully written into /root/.mase/top/hardware/rtl/fc1_weight_source.sv
DEBUG    Init data weight successfully written into /root/.mase/top/hardware/rtl/fc1_weight_rom.dat
DEBUG    Emitting DAT file for node: fc1, parameter: bias
DEBUG    ROM module bias successfully written into /root/.mase/top/hardware/rtl/fc1_bias_source.sv
DEBUG    Init data bias successfully written into /root/.mase/top/hardware/rtl/fc1_bias_rom.dat


1 in emit_bram_transform_pass
/root/.mase/top/hardware/rtl
param_name in emit_bram_handshake: weight
out_size: 4
out_size in emit_parameters_in_mem_internal: 4
fc1
verilog_param_name weight
total_size: 16
shape: [4, 4]
 node.meta[mase].parameters[hardware][verilog_param] {'DATA_IN_0_PRECISION_0': 8, 'DATA_IN_0_PRECISION_1': 5, 'DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'DATA_IN_0_PARALLELISM_DIM_0': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_1': 1, 'DATA_IN_0_PARALLELISM_DIM_1': 1, 'WEIGHT_PRECISION_0': 8, 'WEIGHT_PRECISION_1': 5, 'WEIGHT_TENSOR_SIZE_DIM_0': 4, 'WEIGHT_PARALLELISM_DIM_0': 2, 'WEIGHT_TENSOR_SIZE_DIM_1': 4, 'WEIGHT_PARALLELISM_DIM_1': 2, 'BIAS_PRECISION_0': 8, 'BIAS_PRECISION_1': 5, 'BIAS_TENSOR_SIZE_DIM_0': 4, 'BIAS_PARALLELISM_DIM_0': 2, 'BIAS_TENSOR_SIZE_DIM_1': 1, 'BIAS_PARALLELISM_DIM_1': 1, 'DATA_OUT_0_PRECISION_0': 8, 'DATA_OUT_0_PRECISION_1': 5, 'DATA_OUT_0_TENSOR_SIZE_DIM_0': 4, 'DATA_OUT_0_PARALLELISM_DIM_0': 2, 'DATA_OUT_0_TENSOR_SIZE_DIM_1': 1, 'DATA_OUT_0_PARALLELISM_DIM_1': 1}
o

In [13]:
mg, _ = emit_cocotb_transform_pass(mg)


INFO     Emitting testbench...


> **_TASK:_** Now, you're ready to launch a simulation by calling the simulate action as follows.

In [14]:
from chop.actions import simulate

simulate(skip_build=False, skip_test=False)

INFO: Running command perl /usr/local/bin/verilator -cc --exe -Mdir /workspace/docs/labs/sim_build -DCOCOTB_SIM=1 --top-module top --vpi --public-flat-rw --prefix Vtop -o top -LDFLAGS '-Wl,-rpath,/usr/local/lib/python3.11/dist-packages/cocotb/libs -L/usr/local/lib/python3.11/dist-packages/cocotb/libs -lcocotbvpi_verilator' -Wno-fatal -Wno-lint -Wno-style --trace-fst --trace-structs --trace-depth 3 -I/root/.mase/top/hardware/rtl -I/workspace/src/mase_components/interface/rtl -I/workspace/src/mase_components/language_models/rtl -I/workspace/src/mase_components/memory/rtl -I/workspace/src/mase_components/vivado/rtl -I/workspace/src/mase_components/convolution_layers/rtl -I/workspace/src/mase_components/cast/rtl -I/workspace/src/mase_components/systolic_arrays/rtl -I/workspace/src/mase_components/scalar_operators/rtl -I/workspace/src/mase_components/transformer_layers/rtl -I/workspace/src/mase_components/common/rtl -I/workspace/src/mase_components/hls/rtl -I/workspace/src/mase_components/v

INFO     Build finished. Time taken: 2.58s


make: Leaving directory '/workspace/docs/labs/sim_build'
sys in simulate.py:  <module 'sys' (built-in)>
cmd: [['/workspace/docs/labs/sim_build/top']]
INFO: Running command /workspace/docs/labs/sim_build/top in directory /workspace/docs/labs/sim_build
     -.--ns INFO     gpi                                ..mbed/gpi_embed.cpp:76   in set_program_name_in_venv        Did not detect Python virtual environment. Using system-wide Python interpreter
     -.--ns INFO     gpi                                ../gpi/GpiCommon.cpp:101  in gpi_print_registered_impl       VPI registered
     0.00ns INFO     cocotb                             Running on Verilator version 5.020 2024-01-01
     0.00ns INFO     cocotb                             Running tests with cocotb v1.8.0 from /usr/local/lib/python3.11/dist-packages/cocotb
     0.00ns INFO     cocotb                             Seeding Python random module with 1741563448
     0.00ns INFO     cocotb.regression                  Found test mase_top_

/workspace/src/mase_cocotb/driver.py:25: DeprecationWarning: This method is now private.
  self._thread = cocotb.scheduler.add(self._send_thread())
/workspace/src/mase_cocotb/monitor.py:27: DeprecationWarning: This method is now private.
  self._thread = cocotb.scheduler.add(self._recv_thread())
/usr/lib/python3.11/tempfile.py:1073: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp6ysg0gff'>
  _warnings.warn(warn_message, ResourceWarning)


- :0: Verilog $finish


INFO     Test finished. Time taken: 39.03s


INFO: Results file: /workspace/docs/labs/sim_build/results.xml


The `simulate` action creates a `dump.vcd` file within the `sim_build` directory, which contains the waveform trace of the simulation. The waveforms can be opened with a viewer like GTKWave.

> **TASK**: Follow the instructions [here](https://gtkwave.sourceforge.net/) to install GTKWave on your platform, then open the generated trace file to inspect the signals in the simulation.

# Main Task

Pytorch has a number of layers which are available to users to define neural network models. At the moment, `emit_verilog` supports generating Verilog for models including Linear layers and the ReLU activation.

> **_MAIN TASK:_** choose another layer type from the [Pytorch list](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity) and write a SystemVerilog file to implement that layer in hardware. Then, change the generated `top.sv` file to inject that layer within the design. For example, you may replace the ReLU activations with [Leaky ReLU](https://pytorch.org/docs/stable/generated/torch.nn.RReLU.html#torch.nn.RReLU). Re-run the simulation and observe the effect on latency and accuracy.